<a href="https://colab.research.google.com/github/traderjoevitamins/librarian/blob/main/librarian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [2]:
# i have no idea what these are for or where they came from
import pkg_resources
import importlib
importlib.reload(pkg_resources)

In [ ]:
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'

In [ ]:
pip install -U scikit-learn


In [ ]:
!pip install tensorflow-text


In [7]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_data_validation as tfdv
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
import tensorflow_hub as hub
print('TF version:', tf.__version__)
print('NLKT version:', nltk.__version__)
print('Hub version:', hub.__version__)
print('TFDV version:', tfdv.version.__version__)


TF version: 2.13.0
NLKT version: 3.8.1
Hub version: 0.14.0
TFDV version: 1.14.0


In [13]:
goodreads = pd.read_csv('/content/finalgoodreads.csv')
nyt = pd.read_csv('/content/final_nyt.csv')

In [1]:
synopsis = goodreads['synopsis']



NameError: ignored

In [25]:
tokenizer = nltk.data.load(synopsis, format = 'text')
tokenizer.tokensize()


URLError: ignored